In [1]:
import pandas as pd   
import matplotlib.pyplot as plt
import time

from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline
from matplotlib import pyplot

from numpy import mean
import pandas as pd

In [30]:
data = pd.read_csv("E:/data_set.csv")
data.shape

(5000, 21)

In [3]:
data.columns

Index(['state', 'account_length', 'area_code', 'phone_number',
       'international_plan', 'voice_mail_plan', 'number_vmail_messages',
       'total_day_minutes', 'total_day_calls', 'total_day_charge',
       'total_eve_minutes', 'total_eve_calls', 'total_eve_charge',
       'total_night_minutes', 'total_night_calls', 'total_night_charge',
       'total_intl_minutes', 'total_intl_calls', 'total_intl_charge',
       'number_customer_service_calls', 'class'],
      dtype='object')

In [4]:
data.describe()

,state,account_length,area_code,phone_number,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,class
count,5000.00000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,25.99840,100.25860,436.911400,2499.500000,0.094600,0.264600,7.755200,180.288900,100.029400,30.649668,...,100.191000,17.054322,200.391620,99.919200,9.017732,10.261780,4.435200,2.771196,1.570400,0.141400
std,14.80348,39.69456,42.209182,1443.520003,0.292691,0.441164,13.546393,53.894699,19.831197,9.162069,...,19.826496,4.296843,50.527789,19.958686,2.273763,2.761396,2.456788,0.745514,1.306363,0.348469
min,0.00000,1.00000,408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.00000,73.00000,408.000000,1249.750000,0.000000,0.000000,0.000000,143.700000,87.000000,24.430000,...,87.000000,14.140000,166.900000,87.000000,7.510000,8.500000,3.000000,2.300000,1.000000,0.000000
50%,26.00000,100.00000,415.000000,2499.500000,0.000000,0.000000,0.000000,180.100000,100.000000,30.620000,...,100.000000,17.090000,200.400000,100.000000,9.020000,10.300000,4.000000,2.780000,1.000000,0.000000
75%,39.00000,127.00000,415.000000,3749.250000,0.000000,1.000000,17.000000,216.200000,113.000000,36.750000,...,114.000000,19.900000,234.700000,113.000000,10.560000,12.000000,6.000000,3.240000,2.000000,0.000000
max,50.00000,243.00000,510.000000,4999.000000,1.000000,1.000000,52.000000,351.500000,165.000000,59.760000,...,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000,1.000000


In [194]:
print('Length of Dataset: ', data.shape[0])
print('Number of Attributes: ', data.shape[1])

Length of Dataset:  5000
Number of Attributes:  16


In [5]:
data.head()

,state,account_length,area_code,phone_number,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,...,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,class
0,16,128,415,2845,0,1,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,35,107,415,2301,0,1,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,31,137,415,1616,0,0,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,35,84,408,2510,1,0,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,36,75,415,155,1,0,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [6]:
data.nunique()

state                              51
account_length                    218
area_code                           3
phone_number                     5000
international_plan                  2
voice_mail_plan                     2
number_vmail_messages              48
total_day_minutes                1961
total_day_calls                   123
total_day_charge                 1961
total_eve_minutes                1879
total_eve_calls                   126
total_eve_charge                 1659
total_night_minutes              1853
total_night_calls                 131
total_night_charge               1028
total_intl_minutes                170
total_intl_calls                   21
total_intl_charge                 170
number_customer_service_calls      10
class                               2
dtype: int64

In [7]:
data.drop('phone_number',axis=1,inplace=True)

In [8]:
data.corr()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,class
state,1.000000,0.010334,0.013858,0.007422,-0.030075,-0.027282,0.007334,-0.008166,0.007334,0.003900,-0.005458,0.003896,0.009538,-0.001699,0.009532,-0.009313,0.009898,-0.009322,-0.012997,0.006946
account_length,0.010334,1.000000,-0.018054,0.014681,-0.006650,-0.014575,-0.001017,0.028240,-0.001019,-0.009591,0.009143,-0.009587,0.000668,-0.007825,0.000656,0.001291,0.014277,0.001292,-0.001445,0.021203
area_code,0.013858,-0.018054,1.000000,0.032107,-0.003006,-0.003399,-0.019118,-0.019314,-0.019119,0.007098,-0.012300,0.007114,0.002084,0.014657,0.002070,-0.004154,-0.013623,-0.004219,0.020921,0.010195
international_plan,0.007422,0.014681,0.032107,1.000000,0.009054,0.011947,0.028138,0.008102,0.028141,0.021235,0.002505,0.021234,-0.026508,0.005623,-0.026510,0.031799,0.004493,0.031751,-0.012974,0.259123
voice_mail_plan,-0.030075,-0.006650,-0.003006,0.009054,1.000000,0.954508,0.002059,-0.001735,0.002056,0.023328,-0.005916,0.023332,0.005524,0.009176,0.005522,0.004116,-0.007525,0.004165,-0.014453,-0.110698
number_vmail_messages,-0.027282,-0.014575,-0.003399,0.011947,0.954508,1.000000,0.005381,0.000883,0.005377,0.019490,-0.003954,0.019496,0.005541,0.002676,0.005535,0.002463,0.000124,0.002505,-0.007086,-0.097633
total_day_minutes,0.007334,-0.001017,-0.019118,0.028138,0.002059,0.005381,1.000000,0.001935,1.000000,-0.010750,0.008128,-0.010760,0.011799,0.004236,0.011783,-0.019486,-0.001303,-0.019415,0.002733,0.207705
total_day_calls,-0.008166,0.028240,-0.019314,0.008102,-0.001735,0.000883,0.001935,1.000000,0.001936,-0.000699,0.003754,-0.000695,0.002804,-0.008308,0.002802,0.013097,0.010893,0.013161,-0.010739,0.016130
total_day_charge,0.007334,-0.001019,-0.019119,0.028141,0.002056,0.005377,1.000000,0.001936,1.000000,-0.010747,0.008129,-0.010757,0.011801,0.004235,0.011785,-0.019490,-0.001307,-0.019419,0.002726,0.207700
total_eve_minutes,0.003900,-0.009591,0.007098,0.021235,0.023328,0.019490,-0.010750,-0.000699,-0.010747,1.000000,0.002763,1.000000,-0.016639,0.013420,-0.016642,0.000137,0.008388,0.000159,-0.013823,0.089288


In [31]:
filtered_data = data

In [32]:
filtered_data.shape

(5000, 21)

In [33]:
filtered_data = data.drop('voice_mail_plan',axis=1,inplace=True)

In [35]:
filtered_data = data.drop('number_vmail_messages',axis=1,inplace=True)

In [37]:
filtered_data = data.drop('total_eve_calls',axis=1,inplace=True)

In [38]:
filtered_data = data.drop('total_night_calls',axis=1,inplace=True)

In [39]:
filtered_data = data.drop('total_intl_calls',axis=1,inplace=True)

In [43]:
data.shape

(5000, 16)

In [44]:
data.columns

Index(['state', 'account_length', 'area_code', 'phone_number',
       'international_plan', 'total_day_minutes', 'total_day_calls',
       'total_day_charge', 'total_eve_minutes', 'total_eve_charge',
       'total_night_minutes', 'total_night_charge', 'total_intl_minutes',
       'total_intl_charge', 'number_customer_service_calls', 'class'],
      dtype='object')

In [60]:
X = data.loc[:, data.columns != 'class']
y = data[['class']]

In [61]:
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.3, random_state=2)
print(trainX.shape)
print(trainy.shape)
print(testX.shape)
print(testy.shape)

(3500, 15)
(3500, 1)
(1500, 15)
(1500, 1)


In [62]:
lr = LogisticRegression(solver='liblinear',random_state=2)

In [63]:
lr.fit(trainX,trainy)

c:\Users\Muzammil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=2, solver='liblinear')

In [74]:
md_proba = lr.predict_proba(testX)

In [75]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8121576213530077

In [56]:
accuracy = lr.score(trainX, trainy)
print("Accuracy of Logistic Regression:",accuracy*100,"%")

Accuracy of Logistic Regression: 86.0 %


## Decision Tree

In [91]:
DT = DecisionTreeClassifier(max_depth=200)
DT.fit(trainX,trainy)

DecisionTreeClassifier(max_depth=200)

In [92]:
md_proba = DT.predict_proba(testX)

In [93]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.7560308819916185

In [95]:
accuracy = DT.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 100.0 %


In [163]:
DT = DecisionTreeClassifier(max_depth=400)
DT.fit(trainX,trainy)

DecisionTreeClassifier(max_depth=400)

In [164]:
md_proba = DT.predict_proba(testX)

In [166]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.7527191309854622

In [167]:
accuracy = DT.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 100.0 %


In [168]:
DT = DecisionTreeClassifier(max_depth=600)
DT.fit(trainX,trainy)

DecisionTreeClassifier(max_depth=600)

In [169]:
md_proba = DT.predict_proba(testX)

In [170]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.7568856390433641

In [171]:
accuracy = DT.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 100.0 %


## Grid Search on Decision Tree

In [86]:
gb_cv = DecisionTreeClassifier()
param_grid = {
    'max_depth': [2, 5, 10, 20, 30, 40, 50,100,200,400,800,1000,1200,1500,1700,2000,2300,2500,3000],
    'criterion' :['gini', 'entropy'],
    'max_features': ['auto', 'sqrt', 'log2'] 
}
grid_search = GridSearchCV(estimator = gb_cv, param_grid=param_grid, cv = 10, verbose = 2)
grid_search.fit(trainX, trainy.values.ravel())
best_grid = grid_search.best_estimator_
print(best_grid)

Fitting 10 folds for each of 114 candidates, totalling 1140 fits
[CV] END .....criterion=gini, max_depth=2, max_features=auto; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, max_features=auto; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, max_features=auto; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, max_features=auto; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, max_features=auto; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, max_features=auto; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, max_features=auto; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, max_features=auto; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, max_features=auto; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, max_features=auto; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, max_features=sqrt; total time=   0.0s
[CV] END .....criterion=gini, max_depth=2, m

In [108]:
DT = DecisionTreeClassifier(max_depth=5, criterion='entropy', max_features='sqrt')
DT.fit(trainX,trainy)

DecisionTreeClassifier(criterion='entropy', max_depth=5, max_features='sqrt')

In [109]:
md_proba = DT.predict_proba(testX)

In [116]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8313688916880934

In [115]:
accuracy = DT.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 92.88571428571429 %


In [196]:
kn = KNeighborsClassifier(n_neighbors=5)
kn.fit(trainX,trainy.values.ravel())

KNeighborsClassifier()

In [197]:
md_proba = kn.predict_proba(testX)
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.6023597523626938

In [198]:
accuracy = kn.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 88.5142857142857 %


In [199]:
kn = KNeighborsClassifier(n_neighbors=50)
kn.fit(trainX,trainy.values.ravel())

KNeighborsClassifier(n_neighbors=50)

In [200]:
md_proba = kn.predict_proba(testX)
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.6045935405282882

In [201]:
accuracy = kn.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 86.28571428571429 %


In [202]:
kn = KNeighborsClassifier(n_neighbors=100)
kn.fit(trainX,trainy.values.ravel())

KNeighborsClassifier(n_neighbors=100)

In [203]:
md_proba = kn.predict_proba(testX)
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.5575005796429399

In [204]:
accuracy = kn.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 86.28571428571429 %


# Random Forest

In [120]:
rf = RandomForestClassifier(max_depth=5,n_estimators=50)
rf.fit(trainX,trainy.values.ravel())

RandomForestClassifier(max_depth=5, n_estimators=50)

In [121]:
md_proba = rf.predict_proba(testX)

In [122]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8908783234303789

In [123]:
accuracy = rf.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 91.88571428571429 %


In [124]:
rf = RandomForestClassifier(max_depth=5,n_estimators=150)
rf.fit(trainX,trainy.values.ravel())

RandomForestClassifier(max_depth=5, n_estimators=150)

In [125]:
md_proba = rf.predict_proba(testX)

In [126]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8933353173847894

In [127]:
accuracy = rf.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 92.2 %


In [128]:
rf = RandomForestClassifier(max_depth=15,n_estimators=150)
rf.fit(trainX,trainy.values.ravel())

RandomForestClassifier(max_depth=15, n_estimators=150)

In [129]:
md_proba = rf.predict_proba(testX)

In [130]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8845230144201321

In [131]:
accuracy = rf.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 99.2 %


In [136]:
rf = RandomForestClassifier(max_depth=8,n_estimators=200)
rf.fit(trainX,trainy.values.ravel())

RandomForestClassifier(max_depth=8, n_estimators=200)

In [137]:
md_proba = rf.predict_proba(testX)
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8854106467430987

In [138]:
accuracy = rf.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 96.08571428571429 %


In [132]:
gb_cv = RandomForestClassifier()
param_grid = {
    'max_depth': [2, 5, 10, 20, 30, 40, 50,60,70,80,90,100],
    'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450,500,550,600,700,800,900,1000
    ] 
}
grid_search = GridSearchCV(estimator = gb_cv, param_grid=param_grid, cv = 3, verbose = 2)
grid_search.fit(trainX, trainy.values.ravel())
best_grid = grid_search.best_estimator_
print(best_grid)

Fitting 3 folds for each of 180 candidates, totalling 540 fits
[CV] END ......................max_depth=2, n_estimators=100; total time=   0.5s
[CV] END ......................max_depth=2, n_estimators=100; total time=   0.4s
[CV] END ......................max_depth=2, n_estimators=100; total time=   0.4s
[CV] END ......................max_depth=2, n_estimators=150; total time=   0.6s
[CV] END ......................max_depth=2, n_estimators=150; total time=   0.5s
[CV] END ......................max_depth=2, n_estimators=150; total time=   0.5s
[CV] END ......................max_depth=2, n_estimators=200; total time=   0.9s
[CV] END ......................max_depth=2, n_estimators=200; total time=   0.6s
[CV] END ......................max_depth=2, n_estimators=200; total time=   0.6s
[CV] END ......................max_depth=2, n_estimators=250; total time=   0.7s
[CV] END ......................max_depth=2, n_estimators=250; total time=   0.8s
[CV] END ......................max_depth=2, n_

In [133]:
rf = RandomForestClassifier(max_depth=40,n_estimators=550)
rf.fit(trainX,trainy.values.ravel())

RandomForestClassifier(max_depth=40, n_estimators=550)

In [134]:
md_proba = rf.predict_proba(testX)
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8834087157534839

In [135]:
accuracy = rf.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 100.0 %


# Gradient Boosting

In [189]:
gb = GradientBoostingClassifier(max_depth=2,n_estimators=50)
gb.fit(trainX,trainy)

c:\Users\Muzammil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(max_depth=2, n_estimators=50)

In [190]:
md_proba = gb.predict_proba(testX)
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8954185714137404

In [191]:
accuracy = gb.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 91.68571428571428 %


In [185]:
gb = GradientBoostingClassifier(max_depth=5,n_estimators=50)
gb.fit(trainX,trainy)

c:\Users\Muzammil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(max_depth=5, n_estimators=50)

In [187]:
md_proba = gb.predict_proba(testX)
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8918126732440279

In [188]:
accuracy = gb.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 96.94285714285714 %


In [182]:
gb = GradientBoostingClassifier(max_depth=6,n_estimators=350)
gb.fit(trainX,trainy)

c:\Users\Muzammil\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(max_depth=6, n_estimators=350)

In [183]:
md_proba = gb.predict_proba(testX)


In [184]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8767211934761621

In [157]:
gb_cv = GradientBoostingClassifier()
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 150, 200, 250, 300,400,500,600,700,800],
    "max_depth":[3,5,8],
}
grid_search = GridSearchCV(estimator = gb_cv, param_grid=param_grid, cv = 3, verbose = 2)
grid_search.fit(trainX, trainy.values.ravel())
best_grid = grid_search.best_estimator_
print(best_grid)

Fitting 3 folds for each of 90 candidates, totalling 270 fits
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.8s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.7s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.8s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=150; total time=   1.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=150; total time=   1.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=150; total time=   1.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   1.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   1.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   1.5s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=250; total time=   2.0s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=250; total time=   1.8s
[CV] END ..learning_rate=0.01, max_depth=3, n_e

In [175]:
gb = GradientBoostingClassifier(learning_rate=0.05,max_depth=5,n_estimators=200)
gb.fit(trainX,trainy.values.ravel())

GradientBoostingClassifier(learning_rate=0.05, max_depth=5, n_estimators=200)

In [176]:
md_proba = gb.predict_proba(testX)

In [177]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8891515065525607

In [179]:
accuracy = gb.score(trainX, trainy)
print("Accuracy :",accuracy*100,"%")

Accuracy : 97.88571428571429 %


# Voting Classifier

In [173]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

In [192]:
r0 = GradientBoostingClassifier(learning_rate= 0.05,max_depth = 5,n_estimators=200)
r1 = GradientBoostingClassifier(max_depth = 6,n_estimators=350)
r2 = RandomForestClassifier(max_depth=40,n_estimators=550)
r3 = RandomForestClassifier(max_depth=8,n_estimators=200)
r4 = RandomForestClassifier(max_depth=5,n_estimators=150)
r5 = RandomForestClassifier(max_depth=15,n_estimators=250)
r6 = RandomForestClassifier(max_depth=40,n_estimators=450)
r7 = RandomForestClassifier(max_depth=8,n_estimators=350)

class_vr = VotingClassifier([('gb1', r0), ('gb2', r1),('gb3', r2),('gb4', r3),('gb5', r4),('rf', r5),('rf1', r6),('rf2', r7)], voting='soft')
class_vr.fit(trainX,trainy.values.ravel())
md_proba = class_vr.predict_proba(testX)

In [193]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.8876254018569338

# Performance on Validation set

In [206]:
rf = RandomForestClassifier(max_depth=5,n_estimators=150)
rf.fit(testX,testy.values.ravel())

RandomForestClassifier(max_depth=5, n_estimators=150)

In [208]:
md_proba = rf.predict_proba(testX)

In [209]:
md_probs = md_proba[:,1]
roc_auc_score(testy,md_probs)

0.9467316789574041